In [196]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.utils import Sequence
import re
import string

In [197]:
df = pd.read_csv("/content/drive/MyDrive/merged_file.csv")
df

,Unnamed: 0,English,Bodo
0,0,"The city is famous for its majestic forts, pal...",बुहुमनां दाब दाब जायगानि दावबायारिफोरनि नोजोर ...
1,1,The City Palace was built by Maharaja Jai Sing...,महाराजा जय सिंह II आ बेनि महलफोरखौ लुदोंमोन आर...
2,2,"The Hawa mahal was built by the Maharaja, Sawa...",हावा हलखौ महाराजा सावाइ प्रताप सिंहवा 1799 माइ...
3,3,The Amber Fort complex has several apartments ...,"अम्बर खरङाव महल, नमा, गोजौवाव गाखोलांग्रा सिरि..."
4,4,The Amber Palace is a classic example of Mugha...,अम्बर महला मुगल आरो हिन्दु बुमिनसारनि मोनसे गो...
...,...,...,...
30355,11625,Treatment of viral hepatitis does not depend o...,भाइरेल हेपाटाइटिसनि फाहामथाया आनजादसालि आनजादन...
30356,11626,Vaccine available against hepatitis A is not c...,महामारिजों सोमोन्दो थानाय आरो बांद्राय खरसानि ...
30357,11627,Vaccine is available against hepatitis B only.,हेपाटाइटिस-बि नि थआखायल’ गोबां भेक्सिन दं।
30358,11628,Viral hepatitis may cause serious illness in p...,भाइरेल हेपाटाइटिसआ गोरबोआव थानाय आइजोखौ गोब्रा...


In [198]:
from sklearn.model_selection import train_test_split

In [199]:
eng = df.English
bod = df.Bodo

In [200]:
eng_train, eng_test, bod_train, bod_test = train_test_split(eng,bod,test_size=0.2)

In [201]:
df ['English']=df ['English'].apply (lambda x: str(x).lower() )
df ['Bodo']=df ['Bodo'].apply (lambda x: str(x).lower() )

In [202]:
df ['English']=df ['English'].apply (lambda x: re.sub ("'", '', x))
df ['Bodo']=df ['Bodo'].apply(lambda x: re.sub ("'", '', x))

In [203]:
exclude = set(string.punctuation) 
# Set of all special characters
# Remove all the special characters 
df ['English']=df ['English'].apply (lambda x: ''.join (ch for ch in x if ch not in exclude))
df ['Bodo']=df ['Bodo'].apply (lambda x:''.join (ch for ch in x if ch not in exclude))

In [204]:
remove_digits = str.maketrans ('','', string.digits) 
df ['English']=df ['English'].apply (lambda x: x. translate (remove_digits))
df ['Bodo']=df ['Bodo'].apply (lambda x: x. translate (remove_digits))
df ['Bodo'] = df ['Bodo'].apply (lambda x: re.sub (" [१२३४५६७८९०]", "", x))

In [205]:
df ['Englsih']=df ['English'].apply (lambda x: x.strip () )
df ['Bodo']=df ['Bodo'].apply (lambda x: x.strip ())
df ['English']=df ['English'].apply (lambda x: re.sub (" +", " ", x))
df ['Bodo']=df ['Bodo'].apply (lambda x: re.sub (" +", " ", x))

In [206]:
df ['Bodo'] = df ['Bodo'].apply (lambda x : 'START_ '+ x + ' _END')

In [207]:
all_eng_words=set()
for eng in df ['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add (word)
all_bodo_words=set() 
for bodo in df['Bodo']:
    for word in bodo.split():
        if word not in all_bodo_words:
            all_bodo_words.add(word)

In [208]:
print(len(all_eng_words))
print(len(all_bodo_words))

38687
94394


In [209]:
input_words = sorted(list (all_eng_words) )
target_words = sorted(list (all_bodo_words))
num_encoder_tokens = len (all_eng_words)
num_decoder_tokens = len (all_bodo_words)
num_encoder_tokens, num_decoder_tokens

(38687, 94394)

In [210]:
num_decoder_tokens+=1
print(num_decoder_tokens)

94395


In [211]:
input_token_index = dict([(word, i+1) for i, word in enumerate (input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate (target_words)])

In [212]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [213]:
eng, bod = df['English'], df['Bodo']
eng_train, eng_test, bod_train, bod_test = train_test_split (eng, bod, test_size = 0.1)
eng_train.shape, eng_test.shape

((27324,), (3036,))

In [214]:
def generate_batch(eng=eng_train, bod=bod_train, batch_size=130, max_length_src=40, max_length_tar=50):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(eng), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(eng[j:j+batch_size], bod[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                    # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_index[word] 
                        # decoder input seq
                    if t > 0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield ([encoder_input_data, decoder_input_data], decoder_target_data)

In [215]:
from keras.layers import Input, LSTM, Dense, Embedding
from keras.models import Model
latent_dim=64
# Encoder
encoder_inputs = Input(shape= (None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim,mask_zero = True) (encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm (enc_emb)
# We discard'encoder outputs' and only keep the states.
encoder_states = [state_h, state_c]

In [216]:
# Set up the decoder, using 'encoder states' as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. we don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM (latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# 'encoder input data' & 'decoder input data' into 'decoder target data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [217]:
##decoder at test time
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2= dec_emb_layer(decoder_inputs) 

# Get the embeddings of the decoder sequence
# To predict the next word in the sequence,set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs = decoder_dense(decoder_outputs) 
# A dense softmax layer to generate prob dist. over the target voc
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [218]:
##Finally, we will generate the output by defining the following function and later calling it.

def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    
    stop_condition = False 
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ''+sampled_char
        
        # Exit condition: either hit max lenath
        # or find stop character.
        if (sampled_char =='_END' or
            len(decoded_sentence) > 50):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [219]:
##return

train_gen = generate_batch(eng_train, bod_train, batch_size = 1)
k=-1

In [220]:
k+=1
(input_seq, actual_output),_ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', eng_train[k:k+1].values[0])
print('Actual Bodo Translation:', bod_train[k:k+1].values[0][6:-4])
print('Predicted Bodo Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input English sentence: redefine your priorities and focus your energy and resources on those priorities
Actual Bodo Translation:  नोंनि गिबिथिफोरखौ गोदानबुंफोर आरो नोंनि गोहो आरो नाइबिजिरनायफोरखौ बैफोर गिबिथिफोराव नोजोरहो। 
Predicted Bodo Translation: centrecentrecentrecentreaidsयाchurch’schurch’samer


In [221]:
##return

val_gen = generate_batch(eng_test, bod_test, batch_size = 1)
k=-1

In [222]:
k+=1
(input_seq, actual_output),_ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', eng_test[k:k+1].values[0])
print('Actual Bodo Translation:', bod_test[k:k+1].values[0][6:-4])
print('Predicted Bodo Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 28ms/step
Input English sentence: in the middle of the courtyard stand temples rameshvara temple which is in the centre veerbhadreshvara temple which is to the right and the parvati temple which is to the left
Actual Bodo Translation:  सिथ्लानि गेजेराव मोनदिरफोर दं रामेश्वार मोनदिर जाय मिरुआव दं बिरभद्रेश्वर मोनदिर जाय आगदायाव दं आरो पारबति मोनदिर जाय आगसियाव दं। 
Predicted Bodo Translation: actioncaseआवattendbrownabsorbscdbidiattend
